In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [7]:
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()

,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Pseudo Label

In [10]:
pseudo = pd.read_csv("../input/30-days-pseudo/submission.csv")[target]
train_pseudo = pd.concat([X, y], axis=1)
test_pseudo = pd.concat([X_test, pseudo], axis=1)
all_pseudo = pd.concat([train_pseudo, test_pseudo]).reset_index(drop=True)

# Optuna

In [11]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [12]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=all_pseudo.iloc[:,:-1], y=all_pseudo[target]):
  """
  """
  param_space = {
               'bootstrap_type': 'Poisson',
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
          'task_type': 'GPU',
          'max_depth':trial.suggest_int('max_depth', 4, 6),
           'learning_rate':trial.suggest_uniform('learning_rate', 1.138e-2, 1.338e-2),
          'n_estimators':N_ESTIMATORS,
        'max_bin':trial.suggest_int('max_bin', 606, 667),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 19, 43),
         'reg_lambda':trial.suggest_uniform('reg_lambda', 8, 16),
            'subsample': trial.suggest_uniform('subsample', 0.8664, 0.9664)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  ctb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  trn_idx = X_tr.index
  val_idx = X_va.index
  
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
  oof_idx = np.array([idx for idx in val_idx if idx < train.shape[0]])
  preds_idx = np.array([idx for idx in val_idx if idx >= train.shape[0]])

  X_train, y_train = all_pseudo.iloc[trn_idx, :-1], all_pseudo[target].iloc[trn_idx]
  #X_train = apply_noise_rn(X_train)

  X_valid, y_valid = all_pseudo.iloc[oof_idx, :-1], all_pseudo[target].iloc[oof_idx]
  #X_test = all_pseudo[scale_features].iloc[preds_idx]
  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_seed'] = inseed

    model = ctb.CatBoostRegressor(**param_space)
    model.fit(
        X_train, 
        y_train,
        eval_set=[(X_valid, y_valid)],
        use_best_model=True,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE
       
    )


    ctb_oof[oof_idx] += model.predict(X_valid) / len(seed_list)
    #lgb_pred[preds_idx-train.shape[0]] += model.predict(X_test) / len(seed_list)

  #elapsed = time.time() - start
  rmse = mean_squared_error(y_valid, ctb_oof[oof_idx], squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [13]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 15)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-22 13:25:51,863] A new study created in memory with name: no-name-d7691837-6c4e-4b82-a5bb-d7580fbbae39


0:	learn: 0.5938359	test: 0.7398290	best: 0.7398290 (0)	total: 14.4ms	remaining: 4m 48s
1000:	learn: 0.5716763	test: 0.7225666	best: 0.7225666 (1000)	total: 6.43s	remaining: 2m 1s
2000:	learn: 0.5666490	test: 0.7188411	best: 0.7188411 (2000)	total: 12.3s	remaining: 1m 51s
3000:	learn: 0.5636132	test: 0.7166979	best: 0.7166979 (3000)	total: 18.6s	remaining: 1m 45s
4000:	learn: 0.5614190	test: 0.7152359	best: 0.7152359 (4000)	total: 25.3s	remaining: 1m 41s
5000:	learn: 0.5597239	test: 0.7141524	best: 0.7141524 (5000)	total: 31.2s	remaining: 1m 33s
6000:	learn: 0.5583984	test: 0.7133502	best: 0.7133502 (6000)	total: 37.4s	remaining: 1m 27s
7000:	learn: 0.5573014	test: 0.7127483	best: 0.7127483 (7000)	total: 43.3s	remaining: 1m 20s
8000:	learn: 0.5563847	test: 0.7122987	best: 0.7122987 (8000)	total: 49.5s	remaining: 1m 14s
9000:	learn: 0.5555709	test: 0.7119351	best: 0.7119351 (9000)	total: 56.4s	remaining: 1m 8s
10000:	learn: 0.5548413	test: 0.7116404	best: 0.7116404 (10000)	total: 1m 2s	

[I 2021-08-22 13:31:10,845] Trial 0 finished with value: 0.71037410373623 and parameters: {'max_depth': 4, 'learning_rate': 0.01269430234571873, 'max_bin': 653, 'min_data_in_leaf': 28, 'reg_lambda': 9.787218153714731, 'subsample': 0.9122591527181017}. Best is trial 0 with value: 0.71037410373623.


0:	learn: 0.5938191	test: 0.7398162	best: 0.7398162 (0)	total: 8.27ms	remaining: 2m 45s
1000:	learn: 0.5700426	test: 0.7214214	best: 0.7214214 (1000)	total: 7.06s	remaining: 2m 14s
2000:	learn: 0.5647411	test: 0.7176170	best: 0.7176170 (2000)	total: 14s	remaining: 2m 6s
3000:	learn: 0.5615739	test: 0.7155143	best: 0.7155143 (3000)	total: 21.1s	remaining: 1m 59s
4000:	learn: 0.5593408	test: 0.7141476	best: 0.7141476 (4000)	total: 28.6s	remaining: 1m 54s
5000:	learn: 0.5576759	test: 0.7132426	best: 0.7132426 (5000)	total: 35.7s	remaining: 1m 47s
6000:	learn: 0.5562792	test: 0.7125920	best: 0.7125916 (5999)	total: 42.7s	remaining: 1m 39s
7000:	learn: 0.5551105	test: 0.7121245	best: 0.7121245 (6999)	total: 49.4s	remaining: 1m 31s
8000:	learn: 0.5540706	test: 0.7117541	best: 0.7117541 (8000)	total: 56.5s	remaining: 1m 24s
9000:	learn: 0.5531114	test: 0.7114652	best: 0.7114652 (9000)	total: 1m 4s	remaining: 1m 18s
10000:	learn: 0.5522312	test: 0.7112440	best: 0.7112440 (10000)	total: 1m 11s	

[I 2021-08-22 13:36:05,515] Trial 1 finished with value: 0.7102981730210084 and parameters: {'max_depth': 5, 'learning_rate': 0.012168326966201095, 'max_bin': 617, 'min_data_in_leaf': 26, 'reg_lambda': 8.288848868268229, 'subsample': 0.9492105525714137}. Best is trial 1 with value: 0.7102981730210084.


...

0:	learn: 0.5938279	test: 0.7398245	best: 0.7398245 (0)	total: 12.6ms	remaining: 4m 11s
1000:	learn: 0.5705254	test: 0.7217957	best: 0.7217957 (1000)	total: 7.25s	remaining: 2m 17s
2000:	learn: 0.5652069	test: 0.7179268	best: 0.7179268 (2000)	total: 15.2s	remaining: 2m 17s
3000:	learn: 0.5620408	test: 0.7157770	best: 0.7157770 (3000)	total: 22.3s	remaining: 2m 6s
4000:	learn: 0.5598185	test: 0.7143836	best: 0.7143836 (4000)	total: 29.2s	remaining: 1m 56s
5000:	learn: 0.5581642	test: 0.7134341	best: 0.7134341 (5000)	total: 36.3s	remaining: 1m 48s
6000:	learn: 0.5568189	test: 0.7127579	best: 0.7127579 (6000)	total: 43.6s	remaining: 1m 41s
7000:	learn: 0.5556886	test: 0.7122687	best: 0.7122687 (7000)	total: 50.9s	remaining: 1m 34s
8000:	learn: 0.5546755	test: 0.7118612	best: 0.7118607 (7998)	total: 58.1s	remaining: 1m 27s
9000:	learn: 0.5537718	test: 0.7115663	best: 0.7115663 (9000)	total: 1m 4s	remaining: 1m 19s
10000:	learn: 0.5529309	test: 0.7113293	best: 0.7113284 (9997)	total: 1m 12s

[I 2021-08-22 14:33:06,974] Trial 13 finished with value: 0.7102881474313177 and parameters: {'max_depth': 5, 'learning_rate': 0.011477127962264397, 'max_bin': 606, 'min_data_in_leaf': 33, 'reg_lambda': 15.977518905229068, 'subsample': 0.9639579999293086}. Best is trial 11 with value: 0.7101568290815834.


0:	learn: 0.5938068	test: 0.7398077	best: 0.7398077 (0)	total: 8.35ms	remaining: 2m 47s
1000:	learn: 0.5695923	test: 0.7211045	best: 0.7211045 (1000)	total: 6.92s	remaining: 2m 11s
2000:	learn: 0.5642366	test: 0.7172653	best: 0.7172653 (2000)	total: 15.1s	remaining: 2m 16s
3000:	learn: 0.5611184	test: 0.7152046	best: 0.7152046 (3000)	total: 22.2s	remaining: 2m 5s
4000:	learn: 0.5589245	test: 0.7138581	best: 0.7138581 (4000)	total: 28.8s	remaining: 1m 55s
5000:	learn: 0.5572740	test: 0.7129920	best: 0.7129920 (5000)	total: 36s	remaining: 1m 47s
6000:	learn: 0.5559366	test: 0.7123866	best: 0.7123866 (5998)	total: 44.1s	remaining: 1m 42s
7000:	learn: 0.5548023	test: 0.7119625	best: 0.7119625 (7000)	total: 50.9s	remaining: 1m 34s
8000:	learn: 0.5537729	test: 0.7116305	best: 0.7116303 (7999)	total: 57.9s	remaining: 1m 26s
9000:	learn: 0.5528318	test: 0.7113620	best: 0.7113620 (9000)	total: 1m 4s	remaining: 1m 19s
10000:	learn: 0.5519672	test: 0.7111839	best: 0.7111836 (9999)	total: 1m 11s	r

[I 2021-08-22 14:37:43,567] Trial 14 finished with value: 0.7102731504585664 and parameters: {'max_depth': 5, 'learning_rate': 0.012971315015920802, 'max_bin': 666, 'min_data_in_leaf': 43, 'reg_lambda': 13.130901249838644, 'subsample': 0.9383525362850756}. Best is trial 11 with value: 0.7101568290815834.


Number of finished trials: 15
Best trial: {'max_depth': 5, 'learning_rate': 0.013197859116004091, 'max_bin': 649, 'min_data_in_leaf': 42, 'reg_lambda': 14.390010371698583, 'subsample': 0.9404996480656921}


In [14]:
study.best_params

{'max_depth': 5,
 'learning_rate': 0.013197859116004091,
 'max_bin': 649,
 'min_data_in_leaf': 42,
 'reg_lambda': 14.390010371698583,
 'subsample': 0.9404996480656921}

# Log

====== Ordinal encoding =========

 0.7106017728195324 no noise ver2

 0.71414426290903 row-wise noise ver1

 0.7144009762376538 random noise ver3
 
 ====== One-hot encoding =========
 
 0.7104558237409161 no noise ver7
 
 0.71027414342552 no noise ver8 (narrow space)
 
 
 